## Welcome to the first-ever (and hopefully last-ever) remote ADA final exam (Fall 2020)

In this exam, you will analyze data from Wikipedia and from the [Wikispeedia](https://dlab.epfl.ch/wikispeedia/play/) game, which you have already encountered in lecture 12 on [“Handling networks”](https://docs.google.com/presentation/d/1h6cIINJ9cNZ-Rtb7SskXrl9Xet5zPUoX2oJlNM0loHQ/edit#slide=id.g464f30ace1_0_92). The rules of the Wikispeedia game are simple:
1. Players are given two Wikipedia articles: a *source* and a *target*.
2. Starting from the source article, the goal is to reach the target, exclusively by following links in the articles encountered along the path.

### The data

The Wikispeedia data comprises ~76K games by human players, thereby capturing human navigation behavior on a very small subset of English Wikipedia comprising ~4.5K articles with a total of ~120K links between them.

The dataset is available in the `data` directory pushed to the same GitHub repo as the exam. Inside the data directory, you will find four files:

####  1. `article_df_task-A.tsv.gz`: A tab-separated file with the following information about Wikipedia articles
- *name*: The name of the Wikipedia article
- *topic*: The top-level topic label for the article (each article is assigned exactly one topic label)
- *content*: The cleaned text of the article (having removed punctuations, stopwords, numbers, and letter casing). It is stored as a space-separated string.

#### 2. `links_task-B.tsv.gz`: A tab-separated file containing Wikipedia links
- *linkSource*: The name of the source Wikipedia article from which the link originates
- *linkTarget*: The name of the target Wikipedia article to which the link points

#### 3.  `paths_df_task-B.tsv.gz`: A tab-separated file with the following information about the played games
- *hashed_ip_address*: Anonymized IP address of the player
- *source*: The name of the Wikipedia article from where the game started
- *target*: The name of the target article that the player was supposed to reach 
- *finished*: Whether the game was successfully finished (whether the player reached the target article)
- *human_path_length*: The number of clicks made by the player, before they either reached the target (in *finished* games) or gave up (in *unfinished* games)
- *shortest_path_length*: The minimum number of clicks required to reach the target article from the source article. Disconnected source-target article pairs have a shortest_path_length of *-1*.
- *duration_in_sec*: The duration of the game in seconds

#### 4.  `paths_df_task-C.tsv.gz`: In continuation to the task-B paths dataframe, this tab-separated file has the following additional information
- *in_degree_target*: In-degree of the target article, that is the number of incoming links that lead to the target article

*Note: In all the aforementioned files, the first line contains the header information. Additionally, article names are URL-encoded. For example, 'Antonín_Dvořák' is stored as 'Anton%C3%ADn_Dvo%C5%99%C3%A1k'. If needed, you can decode the article names using this code: `from urllib.parse import unquote; unquote('article-name')`.*

In [ ]:
!pip install sklearn

In [ ]:
''' Some minimalistic imports '''
import pandas as pd
import random
import gzip
import operator
import time
from urllib.parse import unquote

import math
import numpy as np
import scipy.stats as stats
import sklearn

import networkx as nx

from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from urllib.parse import unquote
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

import scipy.stats as stats

# Task A. Can we predict article topic using textual content? (30 points)

*Note-1: For the entire Task A, we will use logistic regression for its scalability and simplicity. Specifically, we will use [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) available in `sklearn` with 'log' loss and '5' epochs for training a logistic regression classifier.*

*Note-2: In all the cases where random number generation is required (train-test split, fitting the model, etc.), make sure to use 42 as the random seed.*

### A1. Load the text data and analyze the topic distribution (2 points)
1. Load `data/article_df_task-A.tsv.gz` as a pandas dataframe.   
   *Note: we have already cleaned the text for you!*
2. Print the total number of unique topics. Additionally, using a plot of appropriate type, analyze the article topic distribution.   
   **/Discuss:/** What do you observe? Are different topics equally likely? If not, is there a large disparity between certain highly popular and very rare topics?

**Comment** Upload the dataframe, check some information about it

In [ ]:
df_article_task_A_original = pd.read_csv("data/article_df_task-A.tsv.gz", # path
                 compression="infer",  # handles compression
                 sep="\t",             # handles .tsv
                 error_bad_lines=False # handles bad lines
)
df_article_task_A_original.describe()

**Comment** Apply the function to have the name more readable

In [ ]:
df_article_task_A_original.name = df_article_task_A.name.apply(lambda x : unquote(x))
df_article_task_A_original.head(5)                                                      

### A2. Topic classification (multi-class): Articles from 5 most frequent topics (8 points)

#### A2.1 Prepping the data for ML (3 points)
1. Restrict the data to articles corresponding to the top-5 topics based on topic frequency.
2. Construct a TF-IDF representation of textual content.
3. Classes are defined by the topic labels, which are given as *strings*. However, for training classifiers via `sklearn` the class labels need to be encoded as ordinals. Perform this encoding and add a new column of ordinal class labels in the current dataframe.
4. Create two numpy arrays: X (TF-IDF representation of each article) and y (class-label of each article).

**A.2.1.1**

In [ ]:
top_5_topics =df_article_task_A_original.groupby(by=['topic']).agg('count').reset_index().sort_values(by='name', ascending=False)['topic'].values[:5]
df_article_task_A_2 = df_article_task_A_original[df_article_task_A_original.topic.isin(top_5_topics)]
df_article_task_A_2.head(5)

**A.2.1.2**

In [ ]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(df_article_task_A_2.content)

Features

In [ ]:
vectorizer.get_feature_names_out()

Shape

In [ ]:
x.shape

**A.2.1.3**

In [ ]:
c = df_article_task_A_2.topic.astype("category")
d = dict(enumerate(c.cat.categories))
df_article_task_A_2['category'] = c.cat.codes
df_article_task_A_2.head(5)

In [ ]:
d

**A.2.1.4**

In [ ]:
X = x.toarray()
y = df_article_task_A_2.category

#### A2.2 Train and evaluate a multi-class classifier (5 points)
1. Perform a train-test split with 70% as the training portion and the remainder as the testing portion. Use `random_state=42` for fixing the seed to 42.
2. Train a logistic regression classifier with L2 regularization using the `SGDClassifier()` method of `sklearn`. Use `SGDClassifier(loss='log', max_iter=5, tol=None, alpha=1e-4, random_state=42)` to initialize the classifier object. Do not modify the settings for any other parameter, let them be set to their default values.   
   **Important heads-up**: *Depending on the configuration of your computer, training the model could take up to 1 minute of compute time.*
3. Report the classifier performance on the test data using accuracy as the metric.
4. **/Discuss:/** Are you satisfied with the model performance? Qualitatively discuss the performance of the model by comparing it to a random baseline and justify your response.



**A.2.2.1**

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.30, random_state=42)

**A.2.2.2**

In [ ]:
clf = SGDClassifier(loss='log', max_iter=5, tol=None, alpha=1e-4, random_state=42)
clf.fit(X_train_2, y_train_2)

**A.2.2.4**

In [ ]:
clf.score(X_test_2, y_test_2)

**Important note:** *Based on the discussion in the class, logistic regression is a binary classification technique. A simple heuristic to perform multi-class classification using a binary classifier is to use the [one-vs.-rest (OVR)](https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-rest) strategy. In this strategy, a binary classifier is trained for each class with the capability to differentiate it from all the other classes, and thus, $k$ binary classifiers are trained for a $k$-class classification. Internally, `SGDClassifier()` uses the [OVR](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html) strategy of `sklearn` whenever it sees the number of unique values in the **y** vector to be greater than 2. Thus, you can simply use it as a black-box! The only important point for this part is to make sure that the **y** vector of class-labels should have **5** unique values, one corresponding to each class-label.*

### A3. Topic classification (binary): Articles from the most and least frequent topics (12 points)

#### A3.1 Train and evaluate a binary classifier (2 points)
1. Restrict the data to articles belonging to the most frequent or to the least frequent topic.
2. Create two numpy arrays: X (TF-IDF representation of each article) and y (class-label of each article).
3. Perform a train-test split with 70% as the training portion and the remainder as the testing portion. Use `random_state=42` for fixing the seed to 42.
4. Train a logistic regression classifier with L2 regularization using the `SGDClassifier()` method of `sklearn`. Use `SGDClassifier(loss='log', max_iter=5, tol=None, alpha=1e-4, random_state=42)` to initialize the classifier object. Do not modify the settings for any other parameter, let them be set to their default values.
5. Report the classifier performance on the test data using accuracy as the metric.

In [ ]:
topics =df_article_task_A_original.groupby(by=['topic']).agg('count').reset_index().sort_values(by='name', ascending=False)['topic'].values
top_and_worst_topics = topics[::len(topics)-1]
df_article_task_A_3 = df_article_task_A_original[df_article_task_A_original.topic.isin(top_and_worst_topics)]

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(df_article_task_A_3.content)

df_article_task_A_3['category'] = df_article_task_A_3.topic.astype("category").cat.codes

X = x.toarray()
y = df_article_task_A_3.category

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

clf = SGDClassifier(loss='log', max_iter=5, tol=None, alpha=1e-4, random_state=42)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

#### A3.2 Assessing the evaluation metric (5 points)
1. **/Discuss:/** What is your take on the accuracy obtained in A3.1? Do you think accuracy is the correct evaluation metric for this task? If yes, justify! If not, why not, and what else can be used?
2. If you included additional evaluation metrics in A3.2.1, then perform an empirical analysis of the performance using these metrics.   
   **/Discuss:/** Additionally, discuss what you observe. Is it harder to predict the topic labels of certain articles? Why, or why not? Please elaborate!

In [ ]:
'''your answer/code here''';

#### A3.3 Class proportions (5 points)
1. **/Discuss:/** Does the disparity in class proportions hurt the model? If yes, how can you fix it? If not, why not? Justify your response with clear and crisp reasons!    
   *Hint: The learning objective of a classifier can be modified by altering the importance of each class in the computation of the loss function.*
2. If your answer to the aforementioned question is a "yes", please do the following. If not, move to **Task A4**.
    * Implement the fix you proposed in A3.3.1, and repeat the classification task performed in Step A3.1 using the fixed model.
    * Vary the regularization parameter `alpha` in the range [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], and using accuracy as the metric find and report its optimal value using 5-fold cross validation on the training set.   
       **Important heads-up**: *Depending on the configuration of your computer, grid search for the optimal value of `alpha` could take up to 1 minute of compute time.*
    * Lastly, using the optimal `alpha` identified in the previous step, redo the evaluations proposed in A3.2.   
       **/Discuss:/** Do you observe any differences from the results obtained in A3.2? Why, or why not? Please elaborate!

In [ ]:
'''your answer/code here''';

### A4. Revisiting Multi-class classifier of A2.2 (4 points)

1. Print and analyze the confusion matrix for the multi-class model trained in A2.2. **/Discuss:/** What do you observe? Please elaborate!
2. **/Discuss:/** Similar to A3.3, does the disparity in class proportions hurt the model? If yes, how can you fix it? If not, why not? Justify your response with clear and crisp reasons!
3. If your answer to the aforementioned question is a "yes", please do the following. If not, move to **Task A5**.
    * Implement the fix you proposed in A4.2, and repeat the classification task performed in Step A2.2 using the fixed model.
    * Vary the regularization parameter `alpha` in the range [1e-6, 1e-5, 1e-4, 1e-3], and using accuracy as the metric find and report its optimal value using 5-fold cross validation on the training set.   
       **Important heads-up**: *Depending on the configuration of your computer, grid search for the optimal value of `alpha` could take up to 2 minutes of compute time.*
    * Lastly, using the optimal `alpha` identified in the previous step, obtain the accuracy and confusion matrix.
4. **/Discuss:/** Contrast the confusion matrix obtained in A4.1 with the one obtained in A4.3. Do you observe any differences? Why, or why not? Please elaborate!

**A.4.1**

In [ ]:
def compute_confusion_matrix(true_label, prediction_proba, decision_threshold=0.5): 
    
    predict_label = (prediction_proba[:,1]>decision_threshold).astype(int)   
                                                                                                                       
    TP = np.sum(np.logical_and(predict_label==1, true_label==1))
    TN = np.sum(np.logical_and(predict_label==0, true_label==0))
    FP = np.sum(np.logical_and(predict_label==1, true_label==0))
    FN = np.sum(np.logical_and(predict_label==0, true_label==1))
    
    confusion_matrix = np.asarray([[TP, FP],
                                    [FN, TN]])
    return confusion_matrix


def plot_confusion_matrix(confusion_matrix):
    [[TP, FP],[FN, TN]] = confusion_matrix
    label = np.asarray([['TP {}'.format(TP), 'FP {}'.format(FP)],
                        ['FN {}'.format(FN), 'TN {}'.format(TN)]])
    
    df_cm = pd.DataFrame(confusion_matrix, index=['Yes', 'No'], columns=['Positive', 'Negative']) 
    
    return sns.heatmap(df_cm, cmap='YlOrRd', annot=label, annot_kws={"size": 16}, cbar=False, fmt='')


def compute_all_score(confusion_matrix, t=0.5):
    [[TP, FP],[FN, TN]] = confusion_matrix.astype(float)
    
    accuracy =  (TP+TN)/np.sum(confusion_matrix)
    
    precision_positive = TP/(TP+FP) if (TP+FP) !=0 else np.nan
    precision_negative = TN/(TN+FN) if (TN+FN) !=0 else np.nan
    
    recall_positive = TP/(TP+FN) if (TP+FN) !=0 else np.nan
    recall_negative = TN/(TN+FP) if (TN+FP) !=0 else np.nan

    F1_score_positive = 2 *(precision_positive*recall_positive)/(precision_positive+recall_positive) if (precision_positive+recall_positive) !=0 else np.nan
    F1_score_negative = 2 *(precision_negative*recall_negative)/(precision_negative+recall_negative) if (precision_negative+recall_negative) !=0 else np.nan

    return [t, accuracy, precision_positive, recall_positive, F1_score_positive, precision_negative, recall_negative, F1_score_negative]

In [ ]:
prediction_proba = clf.predict_proba(X_test_2)
confusion_matrix_05 = compute_confusion_matrix(y_test_2, prediction_proba, 0.5 )
plt.figure(figsize = (4,3)) 
ax = plot_confusion_matrix(confusion_matrix_05)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Confusion matrix for a 0.5 threshold')

### A5. Interpretability (4 points)
1. For each of the 5-classes considered in **A4**, obtain a list of top-10 words (sorted in descending order of their importance) based on the feature weights learned by the classifier.

2. **/Discuss:/** Qualitatively discuss the relevance of the identified words and their association with the respective class-labels. Is the classifier correct to assign them high weights? Are they truly discriminative/predictive of the respective class-labels? Justify your responses with clear and crisp reasons!

In [ ]:
features = vectorizer.get_feature_names_out()
weights = clf.coef_

In [ ]:
df = pd.DataFrame(weights, columns = features)
df = df.T
df.columns = d.values()

In [ ]:
for x in d.values():
    top_10_words = list(df.sort_values(by=x, ascending=False)[x].head(10).index)
    print('Topic: ',x)
    print(top_10_words)

# Task B. The Wikipedia link network (30 points)

### B1. Wikipedia network: reveal your shape! (12 points)
#### B1.1 Load the Wikipedia link network (`data/links_task-B.tsv.gz`) into memory as a directed graph using `NetworkX`. (4 points)
1. Compute the following from the loaded graph object:
    * the number of nodes,
    * the number of edges, and
    * the average degree.
2. **Discuss:** In order to summarize the degree distribution in a single number, would you recommend using the average degree? Why, or why not? If not, what alternatives can you think of? Please elaborate!

Load dataset

In [ ]:
links_task_B = pd.read_csv("data/links_task-B.tsv.gz", # path
                 compression="infer",  # handles compression
                 sep="\t",             # handles .tsv
                 error_bad_lines=False # handles bad lines
)
links_task_B.linkSource = links_task_B.linkSource.apply(lambda x : unquote(x))
links_task_B.linkTarget = links_task_B.linkTarget.apply(lambda x : unquote(x))
links_task_B.head(5)

Create directed graph 

In [ ]:
G=nx.from_pandas_edgelist(links_task_B, source ='linkSource', target = 'linkTarget', create_using=nx.DiGraph())

Informations about the graph

In [ ]:
print('Number of nodes: ', len(G.nodes))
print('Number of edges: ', len(G.edges))
print('AVG degree: ', np.mean(list([x[1] for x in G.degree()])))
print('AVG indegree: ', np.mean(list([x[1] for x in G.in_degree()])))

**Comment** 

First we could look at the average in-degree of each node. However we need to check if the mean is a relevant measure. Indeed if the graph is sparse, some nodes will have more in or out degree compared to others and so the mean is not a good measure. We could use the median, the centrality score also.  

In [ ]:
print('Median degree: ', np.median(list([x[1] for x in G.degree()])))
print('std degree: ', np.std(list([x[1] for x in G.degree()])))

We can see that the median is bigger but also that the std is really important. It means that not all the nodes have the same number of neighbors.

#### B1.2 Using the appropriate plot type, plot the in-degree and out-degree distributions of the Wikipedia link network and analyze it. (4 points)
1. **/Discuss:/** What is the appropriate scale of the axes?
2. **/Discuss:/** Does the distribution (roughly) follow a particular distribution, and if yes, which one? Additionally, explain how you arrived at your conclusion.

In [ ]:
# Helper function for plotting the degree distribution of a Graph
def plot_in_degree_distribution(G):
    degrees = {}
    for node in G.nodes():
        degree = G.in_degree(node)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += 1
    sorted_degree = sorted(degrees.items())
    deg = [k for (k,v) in sorted_degree]
    cnt = [v for (k,v) in sorted_degree]
    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')
    plt.title("In-Degree Distribution")
    plt.ylabel("Frequency")
    plt.xlabel("Degree")
    ax.set_xticks([d+0.05 for d in deg])
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xticklabels(deg)

In [ ]:
# Helper function for plotting the degree distribution of a Graph
def plot_out_degree_distribution(G):
    degrees = {}
    for node in G.nodes():
        degree = G.out_degree(node)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += 1
    sorted_degree = sorted(degrees.items())
    deg = [k for (k,v) in sorted_degree]
    cnt = [v for (k,v) in sorted_degree]
    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')
    plt.title("Out-Degree Distribution")
    plt.ylabel("Frequency")
    plt.xlabel("Degree")
    ax.set_xticks([d+0.05 for d in deg])
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_xticklabels(deg)

In [ ]:
print(nx.info(G))
plot_out_degree_distribution(G)
plot_in_degree_distribution(G)

**Comment** 

1. Since some numbers have a lot of in/out degree and others only a few, it is better to use a loglog scale to visualize the distribution. 
2. The distribution of the in-degree follow a power law as we can see on the bar plot with a loglog scale (can have almost a slop). From the graph of the out degree we can see that the distribution follows a Log-normal distribution. 

#### B1.3 Connectedness of the Wikipedia link network (4 points)

**Definition-1:** An undirected graph $G$ is said to be connected if, for every pair of vertices $(u, v)$, it contains a path that connects the two vertices.

**Definition-2:** A directed graph $G_{dir}$ is **weakly connected** (or simply connected) if the underlying undirected graph obtained by replacing all directed edges of the graph with undirected edges is a connected graph (cf. Definition-1). 

**Definition-3:** A directed graph $G_{dir}$ is **strongly connected** if, for every pair of vertices $(u, v)$, it contains a directed path from $u$ to $v$ and a directed path from $v$ to $u$.

Given these definitions, what can you say about the connectedness of the Wikipedia link network? Specifically, answer the following:
1. Is the Wikipedia link network weakly connected? If not, print the number of weakly connected components. Additionally, print the number of nodes and edges of the subgraph corresponding to the largest weakly connected component.
2. Is the Wikipedia link network strongly connected? If not, print the number of strongly connected components. Additionally, print the number of nodes and edges of the subgraph corresponding to the largest strongly connected component.

**B1.3.1**

In [ ]:
G_undirected = G.to_undirected()
print('- It is '+str(nx.is_connected(G_undirected))+' that the graph is weakly connected.')
print('- There are '+str(nx.number_connected_components(G_undirected))+' weakly connected components.')

c = max(nx.connected_components(G_undirected), key=len)
G_ = G_undirected.subgraph(c).copy()
print('- The biggest weakly connected component has '+str(len(G_.nodes()))+' nodes and '+str(len(G_.edges()))+' edges.')

**B1.3.2**

In [ ]:
G_scc = nx.strongly_connected_components(G)
max_scc = max(G_scc, key=len)
G_max_scc = G.subgraph(max_scc).copy()

print('- It is '+str(nx.is_isomorphic(G, G_max_scc))+' that the graph is strongly connected.')
print('- There are '+str(len(sorted(nx.strongly_connected_components(G), key=len, reverse=True)))+' strongly connected components.')
print('- The biggest strongly connected component has '+str(len(G_max_scc.nodes()))+' nodes and '+str(len(G_max_scc.edges()))+' edges.')

### B2. The interplay between human navigation behavior and Wikipedia network structure (18 points)

#### B2.1 Loading and analyzing the game data (3 points)
1. Load `data/paths_df_task-B.tsv.gz` as a pandas dataframe.
2. Some games were successfully finished (*finished=True*) while others were unsuccessful (i.e., the player gave up before reaching the target article). However, not all unsuccessful games are infeasible; the player might simply not have been good enough at the game to reach the target from the source. **/Discuss:/** Clearly state the reasons why there may be games that are *truly infeasible*.   
3. List all unsuccessful games that are *truly infeasible* and remove them from the dataframe.

**B.2.1.1**

In [ ]:
df_article_task_B_original = pd.read_csv("data/paths_df_task-B.tsv.gz", # path
                 compression="infer",  # handles compression
                 sep="\t",             # handles .tsv
                 error_bad_lines=False # handles bad lines
)
df_article_task_B_original.source = df_article_task_B_original.source.apply(lambda x : unquote(x))
df_article_task_B_original.target = df_article_task_B_original.target.apply(lambda x : unquote(x))
df_article_task_B_original.head(5)

**B.2.1.2**

**Comment** We have seen in part B.1 that the graph is not strongly connected and also not weakly connected. It means that some paths doesn't exist at all in the graph.

**B.2.1.3**

**Comment** There are 24'875 unsuccessful games which represents approximately 32% of all the games.

In [ ]:
unsuccessful = df_article_task_B_original[df_article_task_B_original.finished == False]
nodes_unsuccessful = list(set(unsuccessful.source.values + unsuccessful.target.values))
unsuccessful

In [ ]:
nodes_unsuccessful = set(list(unsuccessful.target.values) + list(unsuccessful.source.values))
nodes_not_in_G = [x for x in nodes_unsuccessful if not G.has_node(x)]

**Comment** Some nodes ar not even in the graph

In [ ]:
unsuccessful_ = unsuccessful[~unsuccessful['source'].isin(nodes_not_in_G)]
unsuccessful_ = unsuccessful_[~unsuccessful_['target'].isin(nodes_not_in_G)]
unsuccessful_

In [ ]:
mask_path =unsuccessful_.apply(lambda x : nx.has_path(G, x.source, x.target),axis=1)
unsuccessful_ = unsuccessful_[mask_path]
unsuccessful_

**Comment** We have removes 36 games that are trully not feasibles

In [ ]:
games_trully_not_feasibles = pd.concat([unsuccessful_,unsuccessful]).drop_duplicates(keep=False)
games_trully_not_feasibles

In [ ]:
successful_ = df_article_task_B_original[df_article_task_B_original.finished == True]
feasibles = pd.concat([unsuccessful_, successful_])

#### B2.2 Wikispeedia => Wikislowia? (9 points)
1. Compute and report the average and median path lengths for games that were successfully finished (*finished=True*). You should compute and report path length in two ways: (1) with respect to the number of clicks needed by *human players*, (2) with respect to the *minimum* number clicks an *optimal player* would need.   
   **/Discuss:/** What can you say about the navigability of the Wikipedia link network?   
   *Hint: Think about [Milgram's experiment](https://docs.google.com/presentation/d/1h6cIINJ9cNZ-Rtb7SskXrl9Xet5zPUoX2oJlNM0loHQ/edit#slide=id.g464f30ace1_0_261) and relate your findings with his findings!*
2. **/Discuss:/** Suppose you want to maliciously decrease Wikipedia's navigability. You are allowed to remove certain edges in the network. What criterion would you use to decide which edges should be removed? Additionally, discuss why these edges are critical for Wikipedia's navigability. Justify your answer with clear and substantive reasoning!   
   _Hint: Navigability of a network depends on the existence of short paths in the network._
3. **/Discuss:/** In continuation to B2.2.2, assume you are given a budget of $k<<n$ edges to be removed. Design the algorithm that you would employ, provide its pseudocode (as a list of steps), and justify your algorithm design choice.   
   *Note: You don't have to describe the steps for computing the criterion you identified in Step B.2.2. Instead, using the identified criterion as a blackbox, you have to provide the steps required to remove $k>1$ edges.*

**B.2.2.1**

In [ ]:
mean_human, median_human = successful_.human_path_length.mean(), successful_.human_path_length.median()
mean_opt, median_opt = successful_.shortest_path_length.mean(), successful_.shortest_path_length.median()
print("For an human, it has the mean is "+str(mean_human)[:4]+" and the median is ",str(median_human))
print("For an optimal path, it has the mean is "+str(mean_opt)[:4]+" and the median is ",str(median_opt))

**Comment** If we look at Milgram's experiment and our results for an human, we can see that we have approimately the same number. we could interpet wikipedia as a small world. It has the same navigability of the world.

**B.2.2.2**

**Comment** Could we centrality measure and remove the ones with the biggest score.

**B.2.2.3**

**Comment** centrality wesh 

#### B2.3 Relationship between network structure and game completion (6 points)
1. Enrich the dataframe with the eigenvector centrality and in-degree of the target article for each game path.
2. Using plots of appropriate type, plot the in-degree distribution of the target articles of game paths. What is the appropriate scale of the axes? Additionally, summarize the in-degree distribution of the target articles in a single number using a suitable metric. Justify your choice of the metric.
3. Using plots of appropriate type, analyze the differences in in-degree and eigenvector centrality of finished and unfinished games.   
   **/Discuss:/** What do you observe? Clearly discuss your observations!

**B.2.3.1**

In [ ]:
centrality = nx.eigenvector_centrality(G)
feasibles['eigen_centrality'] = feasibles.target.apply(lambda x : centrality[x])
feasibles['in_degree'] = feasibles.target.apply(lambda x : G.in_degree(x))
feasibles.head(3)

**B.2.3.2**

In [ ]:
fig, ax = plt.subplots()
feasibles.in_degree.hist(ax=ax, bins=100)

In [ ]:
feasibles.hist(column='in_degree', by='finished', sharex=True, sharey=True)

In [ ]:
feasibles.hist(column='eigen_centrality', by='finished', sharex=True, sharey=True)

# Task C. Putting on the causality hat (30 points)

### C1. Are articles with high in-degree easier to find? A naive analysis (6 points)

Recall that in each Wikispeedia game, a player is **randomly** assigned a source article and a target article. In task **C**, we are interested in measuring the effect of the target article's in-degree on the player's chances of successfully finishing the game.
1. Load `data/paths_df_task-C.tsv.gz` as a pandas dataframe.
2. Consider target articles with fewer than 20 incoming links to have a *low in-degree* and articles with at least 20 to have a *high in-degree*.
3. Calculate the fraction of games with a high-in-degree target that was finished, and the fraction of games with a low-in-degree target that was finished. Do you observe significant differences, and in what direction? (You may do a statistical test or provide confidence intervals.)
4. Repeat the same analysis for the game duration, for finished games only.
5. Quantify and discuss your findings from Steps C1.3 and C1.4.

In [ ]:
df = pd.read_csv("data/paths_df_task-C.tsv.gz", # path
                 compression="infer",  # handles compression
                 sep="\t",             # handles .tsv
                 error_bad_lines=False # handles bad lines
                           )
df.head(5)

In [ ]:
df['low_in'] = df.in_degree_target.apply(lambda x : int(x < 20))
df['high_in'] = df.in_degree_target.apply(lambda x : int(x >= 20))

In [ ]:
df_finished_low = df[(df.low_in ==1) &(df.finished == True)]
df_finished_high = df[(df.high_in ==1) &(df.finished == True)]
f_finished_low = df_finished_low.shape[0]/(df[df.low_in ==1].shape[0])
f_finished_high = df_finished_high.shape[0]/(df[df.high_in ==1].shape[0])

In [ ]:
print('Proportion of games finished with low in-degree: %f'% (100*f_finished_low))
print('Proportion of games finished with high in-degree: %f'% (100*f_finished_high))

**Comment** 

we can see that there is more games that were finished when the target has a high-degree. 

Let's look at the statistics. Determine whether the difference between the two categories is statistically significant.

In [ ]:
stats.ttest_ind(df[df.low_in ==1].finished, df[df.high_in ==1].finished)

**Comment C.1.3**

Since the p-value is 0.0, we can conclude that the difference between the two categories is statistically significant.

In [ ]:
df_finished = df[df.finished == True]
df_finished_low = df_finished[df_finished.low_in ==1]
df_finished_high = df_finished[df_finished.high_in ==1]

In [ ]:
stats.ttest_ind(df_finished_high.duration_in_sec, df_finished_low.duration_in_sec)

**Comment C.1.4**

Since the p-value is high, we can conclude that the difference between the two categories is not statistically significant and then we can not reject the null hypothesis

**Comment C.1.5**

If we look at the games that were finished, we can notice that the in-degree of the target is not relevant with the point C.1.4 and so if we finish, the time is quite the same. however the proportion of people who are able to finish the games is bigger with high in-degree. This analysis is relvant for the number of in-degree (finished or not) but not for the time it takes to finish.

### C2. Modeling the study setup with a causal diagram (9 points)

**C2.1** We are interested in the effect of target in-degree (**deg(T)**) on the binary outcome **(O)** that captures whether the game was successfully finished or not. In the previously conducted naive analysis, we did not take into account the shortest path length **(L)**, which is the minimum number of clicks necessary in order to reach the target article from the source article **(S)**. Target and source articles are randomly assigned, independently from each other. The assigned source article S and deg(T) affect the shortest path length L. The source article affects the outcome through the shortest path length, and also directly, as some sources might be more central in the network.   
Choose a causal diagram **(A, B, C, D, E, or F)** that corresponds to the described relationships between the following 4 variables. Additionally, justify your choice and explain your reasoning. **(4 points)**
- **deg(T):** target in-degree,
- **L:** shortest path length between source and target,
- **S:** name of source article,
- **O:** outcome, i.e., the binary indicator that says whether the game was successfully finished.

In [ ]:
Image(filename="img/diagram.png") 

**Comment C.2.1**

We should choose the diagram B. Indeed deg(T) and S are indenpendent and we should not have an arrow between them. The shortest path lenght should only have an arrow from the deg(T) and S since they influcence it but not the outcome O. Now the outcome O is affected by the deg(T), shortest path length and the source S. 

**C2.2** 

What is potentially misleading about the naive analysis done in task **C1**? In what conceivable ways could the conclusions about the impact of target in-degree **deg(T)** on the outcome **O** be wrong?
Could the findings go in the opposite direction of a true direct causal effect? Justify your answer. **(5 points)**

**Comment C.2.2**

The deg(T) doesn't not only affect the outcome O directly but also throught the shortest path length. Also the outcome O depends on the source (position). We are missing an important variable in the outcome O that depends on both deg(T) and S.

### C3. A more controlled analysis: matching on source article and shortest path length (15 points)

Let's now understand how the estimated effect of the target in-degree would differ if we controlled for both the source article and the shortest path length between source and target. Construct a set of matched pairs of games. In a matched pair, the two games differ in target in-degree (one has a high in-degree, and the other has a low in-degree), but the source article and the shortest path length are exactly the same. Note that your output must be a 1-to-1 matching.

1. Perform exact matching on source article and shortest path length. This means that two candidate games can be matched only if the source article and the shortest path length are exactly the same. Additionally, report the number of matched pairs of games. **(7 points)**   
   **Important heads-up**: *Depending on the configuration of your computer, computing the exact matching could take up to 3 minutes of compute time.*
2. Among the matched pairs of games, calculate the fraction of games with a high-in-degree target that was successfully finished, and the fraction of games with a low-in-degree target that was successfully finished. Do you observe significant differences, and in what direction? (You may do a statistical test or provide confidence intervals.) **(3 points)**
3. **/Discuss:/** What do you observe? Are the conclusions different from the conclusions reached in C1? If yes, speculate as to why that is the case. How is the estimation in task C1 different from the estimation in task C3? **(4 points)**
4. **/Discuss:/** Based on your findings, how should Wikipedia be modified in order to increase the fraction of finished Wikispeedia games? **(1 points)**

*Hint: How you construct the set of matched pairs is up to you. One way can be to build an unweighted bipartite graph where games with targets with a high in-degree are on one side (the "treated class"), and games with targets with a low in-degree are on the other side (the "control class"). The edge between two matching candidate games is created only if there is an exact match on the source article and the shortest path length. Based on this bipartite graph, you can then find the **optimal** matching. However, note that this is neither the required nor the only way.*

**Code C.3.1**

In [ ]:
df.head(5)

In [ ]:
df_low = df[df.low_in ==1]
df_high = df[df.high_in ==1]
df_low

In [ ]:
res = df_low.merge(df_high, how='inner', on=['source','shortest_path_length'])
res = res[['source','shortest_path_length', 'target_x', 'finished_x','target_y', 'finished_y']]
res = res.drop_duplicates()
res

In [ ]:
source = list(res.source.unique())
matched_x = list(res.target_x.unique())
matched_y = list(res.target_y.unique())
matched = matched_x + matched_y

In [ ]:
df = df[(df.source.isin(source)) & df.target.isin(matched)]

In [ ]:
df['low_in'] = df.in_degree_target.apply(lambda x : int(x < 20))
df['high_in'] = df.in_degree_target.apply(lambda x : int(x >= 20))
df_finished_low = df[(df.low_in ==1) &(df.finished == True)]
df_finished_high = df[(df.high_in ==1) &(df.finished == True)]
f_finished_low = df_finished_low.shape[0]/(df[df.low_in ==1].shape[0])
f_finished_high = df_finished_high.shape[0]/(df[df.high_in ==1].shape[0])

In [ ]:
print('Proportion of games finished with low in-degree: %f'% (100*f_finished_low))
print('Proportion of games finished with high in-degree: %f'% (100*f_finished_high))